<a href="https://colab.research.google.com/github/omartafsi/memory-based-collaborative-filtering-demo/blob/main/memory_based_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
user_item_matrix = pd.read_csv("movie_rating.csv")
user_item_matrix.index.name = "user_id"

In [ ]:
user_item_matrix

,movie_1,movie_2,movie_3,movie_4,movie_5,movie_6,movie_7,movie_8,movie_9,movie_10
user_id,,,,,,,,,,
0,1.0,NaN,5.0,5.0,5.0,4.0,3.0,2.0,NaN,4.0
1,2.0,4.0,5.0,4.0,4.0,2.0,4.0,3.0,3.0,5.0
2,5.0,NaN,NaN,5.0,4.0,5.0,4.0,1.0,4.0,2.0
3,3.0,NaN,1.0,3.0,2.0,5.0,1.0,NaN,1.0,2.0
4,1.0,1.0,1.0,1.0,5.0,NaN,1.0,2.0,4.0,1.0
...,...,...,...,...,...,...,...,...,...,...
395,3.0,1.0,2.0,3.0,2.0,5.0,NaN,NaN,2.0,2.0
396,2.0,5.0,4.0,3.0,4.0,5.0,4.0,5.0,2.0,5.0
397,2.0,5.0,NaN,2.0,NaN,2.0,NaN,1.0,NaN,NaN


In [ ]:
user_item_matrix.fillna(0,inplace=True)

In [ ]:
# Calculate cosine similarity between users
user_similarity = cosine_similarity(user_item_matrix)

In [ ]:
# Choose a target user for recommendations
target_user_id = 0
# Find the most similar users to the target user
similar_users = user_similarity[target_user_id]

In [ ]:
# Find the indices of the top-k most similar users
k = 3  # Number of similar users to consider
top_k_users_indices = similar_users.argsort()[-k - 1:-1][::-1]
print(f"The most similar users to target_user_id  {target_user_id} are : {', '.join([ str(id) for id in top_k_users_indices.tolist()])}")

The most similar users to target_user_id  0 are : 310, 209, 15


In [ ]:
# get coefficient of top k similar users and normalize them
similarity_scores = np.sort(similar_users)[-k - 1:-1][::-1]
normalized_similarity_scores = similarity_scores/similarity_scores.sum()
print(f"the normalised similarity scores of the {k} users :  {', '.join([ str(round(id,3)) for id in normalized_similarity_scores.tolist()])}")

the normalised similarity scores of the 3 users :  0.34, 0.33, 0.33


In [ ]:
# Get the ratings of the target user
target_user_ratings = user_item_matrix.iloc[target_user_id]

# Predict ratings for items the target user hasn't rated
predicted_ratings = user_item_matrix.iloc[top_k_users_indices].mul(normalized_similarity_scores, axis=0).loc[::,target_user_ratings == 0].dropna()

print(f"""the predicted scores for user {target_user_id} are : \n{predicted_ratings.sum().round()}""")

the predicted scores for user 0 are : 
movie_2    0.0
movie_9    2.0
dtype: float64
